In [15]:
!ls /workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/

df_model.onnx  df_model_fifos.onnx  df_model_ooc.onnx  df_model_stitchedip.onnx


In [1]:
!ls /workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/

10_step_resnet50_slr_floorplan.onnx  dataflow_partition20_egc7rr3f
dataflow_partition0_bdji84tj	     dataflow_partition21_i0o2uc8q
dataflow_partition10_594dwkrc	     dataflow_partition22_61jgvirb
dataflow_partition11_vqbfb73w	     dataflow_partition2_wv9k2ync
dataflow_partition12_r4b53u14	     dataflow_partition3_f5lzgb5p
dataflow_partition13_y7wfc5b7	     dataflow_partition4_vsff984h
dataflow_partition14_17svg8jd	     dataflow_partition5_pt1gyz4f
dataflow_partition15_9i6bapcs	     dataflow_partition6_m4ytq5q4
dataflow_partition16_aorcr0ns	     dataflow_partition7_jmbmsu6v
dataflow_partition17_21iblt85	     dataflow_partition8_zb_hsgbj
dataflow_partition18_9nin_8db	     dataflow_partition9_dcl3wncn
dataflow_partition19_negmuo1f	     df_model_bitfile.onnx
dataflow_partition1_sayfzn23


In [18]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model_ooc.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model_ooc.onnx' at http://0.0.0.0:8081


In [6]:
from finn.core.modelwrapper import ModelWrapper
from finn.analysis.partitioning import partition
import numpy as np
from finn.transformation.general import ApplyConfig

model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/10_step_resnet50_slr_floorplan.onnx")
# apply partitioning of the model, restricting the first and last layers to SLR0
default_slr = 0
abs_anchors = [(0,[default_slr]),(-1,[default_slr])]
#increase resource limits to make partitioning feasible, except for SLR0 which also has DDR subsystem
limits = np.array([[0.75,.5,.7,.6,.6],[1,.7,.9,.8,.8],[1,.7,.9,.8,.8],[1,.7,.9,.8,.8]])
#floorplan = partition(model, 4, "U250", abs_anchors=abs_anchors, multivariant=False, linear_cuts=True, limits=limits)[0]
floorplan = partition(model, 4, "U250", abs_anchors=abs_anchors, multivariant=False, linear_cuts=True, limits=limits)
# apply floorplan to model
#model = model.transform(ApplyConfig(floorplan))
#print("SLR floorplanning applied")

#model.save("/tmp/resnet50_floorplan.onnx")


In [7]:
print(floorplan)

None


In [2]:
from finn.util.visualization import showInNetron

In [3]:
showInNetron("/workspace/results/ooc_vivado_log/baseline/old_model.onnx")

Serving '/workspace/results/ooc_vivado_log/baseline/old_model.onnx' at http://0.0.0.0:8081


In [12]:
from finn.core.modelwrapper import ModelWrapper

model = ModelWrapper("/workspace/results/ooc_vivado_log/baseline/old_model.onnx")

list_of_names=[]
for n in model.graph.node:
    list_of_names.append(n.name)
    
list_of_names

['StreamingFIFO_0',
 'StreamingFCLayer_Batch_0',
 'StreamingFIFO_1',
 'DuplicateStreams_Batch_0',
 'StreamingFIFO_2',
 'StreamingFIFO_3',
 'StreamingDataWidthConverter_Batch_0',
 'FMPadding_Batch_0',
 'StreamingFIFO_4',
 'StreamingFIFO_5',
 'StreamingFCLayer_Batch_1',
 'ConvolutionInputGenerator1D_0',
 'StreamingFIFO_6',
 'StreamingFIFO_7',
 'Vector_Vector_Activate_Batch_0',
 'StreamingFIFO_8',
 'StreamingDataWidthConverter_Batch_1',
 'StreamingFIFO_9',
 'StreamingFCLayer_Batch_2',
 'StreamingFIFO_10',
 'FMPadding_Batch_1',
 'StreamingFIFO_11',
 'ConvolutionInputGenerator1D_1',
 'StreamingFIFO_12',
 'Vector_Vector_Activate_Batch_1',
 'StreamingFIFO_13',
 'StreamingDataWidthConverter_Batch_2',
 'StreamingFIFO_14',
 'StreamingFCLayer_Batch_3',
 'StreamingFIFO_15',
 'FMPadding_Batch_2',
 'StreamingFIFO_16',
 'ConvolutionInputGenerator1D_2',
 'StreamingFIFO_17',
 'Vector_Vector_Activate_Batch_2',
 'StreamingFIFO_18',
 'StreamingDataWidthConverter_Batch_3',
 'StreamingFIFO_19',
 'StreamingF

In [76]:
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp

max_cyc = 0
model = ModelWrapper("/workspace/results/syn_end2end_2806/df_model.onnx")
for n in model.graph.node:
    inst = getCustomOp(n)
    if n.op_type in ["FMPadding_Batch", "DuplicateStreams_Batch", "Thresholding_Batch", "AddStreams_Batch"]:
        cyc_est = inst.get_nodeattr("cycles_estimate")
        if cyc_est>4000000:
            print("FOUND")
    if n.op_type=="ConvolutionInputGenerator1D":
        cyc_est = inst.get_nodeattr("cycles_estimate")
        if cyc_est>4000000:
            print("FOUND")
    if n.op_type=="Vector_Vector_Activate_Batch":
        cyc_est = inst.get_nodeattr("cycles_estimate")
        if cyc_est>4000000:
            print("FOUND")
    if n.op_type=="StreamingFCLayer_Batch":
        cyc_est = inst.get_nodeattr("cycles_estimate")
        if cyc_est>4000000:
            print(n.name)
#model = model.transform(SetFolding(target_cycles_per_frame=8400000))

StreamingFCLayer_Batch_1
StreamingFCLayer_Batch_2
StreamingFCLayer_Batch_3
StreamingFCLayer_Batch_4
StreamingFCLayer_Batch_5
StreamingFCLayer_Batch_6
StreamingFCLayer_Batch_7
StreamingFCLayer_Batch_8
StreamingFCLayer_Batch_9
StreamingFCLayer_Batch_10
StreamingFCLayer_Batch_11
StreamingFCLayer_Batch_12
StreamingFCLayer_Batch_13
StreamingFCLayer_Batch_14
StreamingFCLayer_Batch_15
StreamingFCLayer_Batch_16
StreamingFCLayer_Batch_17
StreamingFCLayer_Batch_18
StreamingFCLayer_Batch_19
StreamingFCLayer_Batch_20
StreamingFCLayer_Batch_21
StreamingFCLayer_Batch_22
StreamingFCLayer_Batch_23
StreamingFCLayer_Batch_24
StreamingFCLayer_Batch_25
StreamingFCLayer_Batch_26
StreamingFCLayer_Batch_27
StreamingFCLayer_Batch_28
StreamingFCLayer_Batch_29
StreamingFCLayer_Batch_30
StreamingFCLayer_Batch_31
StreamingFCLayer_Batch_32
StreamingFCLayer_Batch_33
StreamingFCLayer_Batch_34
StreamingFCLayer_Batch_35
StreamingFCLayer_Batch_36
StreamingFCLayer_Batch_37
StreamingFCLayer_Batch_38
StreamingFCLayer_Batc

In [7]:
showInNetron("/workspace/results/quartznet_export.onnx")

Serving '/workspace/results/quartznet_export.onnx' at http://0.0.0.0:8081


In [6]:
!ls /workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/

dataflow_partition0_bdji84tj   dataflow_partition20_egc7rr3f
dataflow_partition10_594dwkrc  dataflow_partition21_i0o2uc8q
dataflow_partition11_vqbfb73w  dataflow_partition22_61jgvirb
dataflow_partition12_r4b53u14  dataflow_partition2_wv9k2ync
dataflow_partition13_y7wfc5b7  dataflow_partition3_f5lzgb5p
dataflow_partition14_17svg8jd  dataflow_partition4_vsff984h
dataflow_partition15_9i6bapcs  dataflow_partition5_pt1gyz4f
dataflow_partition16_aorcr0ns  dataflow_partition6_m4ytq5q4
dataflow_partition17_21iblt85  dataflow_partition7_jmbmsu6v
dataflow_partition18_9nin_8db  dataflow_partition8_zb_hsgbj
dataflow_partition19_negmuo1f  dataflow_partition9_dcl3wncn
dataflow_partition1_sayfzn23   df_model_bitfile.onnx


In [8]:
showInNetron("/workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/df_model_bitfile.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/df_model_bitfile.onnx' at http://0.0.0.0:8081


In [11]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/dataflow_partition16_aorcr0ns/df_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/dataflow_partition16_aorcr0ns/df_model.onnx' at http://0.0.0.0:8081


In [12]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP

model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model_stitchedip.onnx")
print(model.get_metadata_prop("vivado_stitch_ifnames"))


model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/dataflow_partition10_594dwkrc/df_model.onnx")
print(model.get_metadata_prop("vivado_stitch_ifnames"))

model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/dataflow_partition16_aorcr0ns/df_model.onnx")
print(model.get_metadata_prop("vivado_stitch_ifnames"))

model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/tmp_experiments/dataflow_partition2_wv9k2ync/df_model.onnx")
print(model.get_metadata_prop("vivado_stitch_ifnames"))



{"clk": ["ap_clk"], "rst": ["ap_rst_n"], "s_axis": [["s_axis_0", 8]], "m_axis": [["m_axis_0", 24]], "aximm": [], "axilite": ["s_axilite_0", "s_axilite_1", "s_axilite_2", "s_axilite_3", "s_axilite_4", "s_axilite_5", "s_axilite_6", "s_axilite_7", "s_axilite_8", "s_axilite_9", "s_axilite_10", "s_axilite_11", "s_axilite_12", "s_axilite_13", "s_axilite_14", "s_axilite_15", "s_axilite_16", "s_axilite_17", "s_axilite_18", "s_axilite_19", "s_axilite_20", "s_axilite_21", "s_axilite_22", "s_axilite_23", "s_axilite_24", "s_axilite_25", "s_axilite_26", "s_axilite_27", "s_axilite_28", "s_axilite_29", "s_axilite_30", "s_axilite_31", "s_axilite_32", "s_axilite_33", "s_axilite_34", "s_axilite_35", "s_axilite_36", "s_axilite_37", "s_axilite_38", "s_axilite_39", "s_axilite_40", "s_axilite_41", "s_axilite_42", "s_axilite_43", "s_axilite_44", "s_axilite_45", "s_axilite_46", "s_axilite_47", "s_axilite_48", "s_axilite_49", "s_axilite_50", "s_axilite_51", "s_axilite_52", "s_axilite_53", "s_axilite_54", "s_ax